# Homework 1: Sentiment Analysis with Naïve Bayes
#### CSCI 3832 Natural Language Processing


1. Lemmas and inflected forms, hyponyms/hypernyms, the distributional hypothesis
2. Tokenization, vocabularies, and feature extraction for a Naive Bayes model 

## Julia Troni
### julia.troni@colorado.edu or jutr6738@colorado.edu


## Section 1: Free Response Questions


**Question 1: Write down the lemmas of the following inflected forms:**
1. walked
2. taught
3. best
4. are
5. running

*Your answer here*
1. walk
2. teach
3. good
4. be
5. run

**Question 2: Write down 3 hyponyms of the following words:**
1. dog
2. food
3. profession

*Your answer here*
1. dog- golden retriever, lab, border collie
2. food - peanut butter, banana, tofu
3. profession- teacher, chef, doctor

**Question 3: In your own words, describe:**
1. The distributional hypothesis (see lecture on distributional semantics)
2. How is the distributional hypothesis relvant to NLP systems?

*Your answer here*

The distributional hypothesis states that words that occur in similar contexts tend to have similar meanings. 
This is relevant to NLP systems because it is the foundation for many techniques, such as word embeddings, which can then be used as input for many NLP applications such as language translation and text classification. Also,the distributional hypothesis can also be used to develop models of word meaning and similarity that can be used in tasks such as word sense disambiguation.

## Section 2: Sentiment Analysis with Naive Bayes

In this section, our goal is to classify a set of movie reviews as positive or negative. For our dataset, we'll use the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). To get started, download the dataset from the link, and extract it to where your notebook is. Next, we'll load the data and look at a couple of examples. 

*Important: for any project which involves creating or training models, you can **only** do your exploratory data analysis on the training set. Looking at the test set in any way can invalidate your results!*

In [1]:
import os

data_dir = 'aclImdb/'

pos_train_dir = data_dir + 'train/pos/'
neg_train_dir = data_dir + 'train/neg/'

def read_folder(folder):
    examples = []
    for fname in os.listdir(folder):
        with open(os.path.join(folder, fname), encoding='utf8') as f:
            examples.append(f.readline().strip())
    return examples

pos_examples = read_folder(pos_train_dir)
neg_examples = read_folder(neg_train_dir)

print('Number of positive examples: {}\nNumber of negative examples: {}\n\n'.format(len(pos_examples), len(neg_examples)))

print('Sample positive example: {}\n\n'.format(pos_examples[0]))
print('Sample negative example: {}'.format(neg_examples[0]))



Number of positive examples: 12500
Number of negative examples: 12500


Sample positive example: Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


Sample negative example: Story of a man who has unnatural feelings for a pig. Starts out with 

Now that we've loaded the data, let's create our vocabulary. While we want our vocabulary to cover the whole training set, we'll keep them separate to see if there are any words which are frequently found in one or the other class -- these words might be informative features for classification! 

The simplest way to create a vocabulary is to split on spaces:

In [2]:
pos_words = []  # A list of all space separated tokens found across all positive examples. (Contains duplicates)
neg_words = []

pos_vocab = set()  # A list of *unique* separated tokens found in across all positive examples. (No duplicates)
neg_vocab = set()

In [3]:
##Counts of total words (tokens)

pos_words=[]  # A list of all space separated tokens found across all positive examples. (Contains duplicates)

for pos in pos_examples:
    strs=pos.split(' ')
    for s in strs:
        pos_words.append(s)

neg_words = []
for neg in neg_examples:
    strs=neg.split(' ')
    for s in strs:
        neg_words.append(s)

#counts of unique words (types)
pos_vocab = set(pos_words)  # A list of *unique* separated TYPES found in across all positive examples. (No duplicates)
neg_vocab = set(neg_words)

In [4]:
# Sanity check

print(len(pos_words))
print(len(pos_vocab))

assert len(pos_words) == 2958696
assert len(pos_vocab) == 178873

2958696
178873


Now lets calculate word frequencies for each class. (Hint: use the Python Counter class)

In [5]:
pos_frequencies = [] # A list of tuples of the form (word, count). 
                 # The list should be sorted in descending order, using the count of each tuple as the key

neg_frequencies = []

In [6]:
from collections import Counter

''' 
Your code here. For each class (positive/negative) calculate the frequency of each word and save it in pos_counter
and neg_counter.

Print the top 15 most common word for each class. 

'''

pos_frequencies = Counter(pos_words).most_common() # A list of tuples of the form (word, count). 
                 # The list should be sorted in descending order, using the count of each tuple as the key

neg_frequencies = Counter(neg_words).most_common() 


In [7]:
assert pos_frequencies[0] == ('the', 148413)
assert neg_frequencies[0] == ('the', 138612)

In [8]:

print("Top 15 positive words", pos_frequencies[0:15])
print('')
print("Top 15 negative words", neg_frequencies[0:15])

Top 15 positive words [('the', 148413), ('and', 84270), ('a', 79427), ('of', 75341), ('to', 65209), ('is', 55358), ('in', 45794), ('that', 31941), ('I', 30927), ('it', 26987), ('this', 26021), ('/><br', 24617), ('as', 23930), ('with', 22031), ('was', 21308)]

Top 15 negative words [('the', 138612), ('a', 75665), ('and', 68381), ('of', 67629), ('to', 67359), ('is', 47870), ('in', 39782), ('I', 35043), ('that', 32615), ('this', 31177), ('it', 27440), ('/><br', 26318), ('was', 25389), ('for', 20197), ('with', 19687)]


Looking at the top 15 words for each class we see two problems:

1. The words are essentially the same for each class, which doesn't give us any information on how to differentiate them.
2. Look at the most frequent tokens. Are there any tokens which aren't words? Any situations where tokens with different surface forms but the same meaning could be repeated (and if so, how might we control for this?)

*Your answer to 2 here*

- Tokens that appear that are not words but still represent meaning: 
    - punctuation like   "...",   "<br",   "br/>",   "/>"   
    - emoji representations such as   XD,   :p,   =],   O_O 
    - numbers 


- Tokens with different surface forms but the same meaning that are repeated: 
    - Uppercase vs. lowercase 
        - And vs. and
        - OK vs. Ok vs. ok
        - We can control for this by converting the list of words to all lowercase using .tolower()
    - With vs. without punctuation and differences in spacing:
        - and...funny vs.  and... vs. funny
 

Instead of looking at the most frequent words, let's instead look at the most frequent words which explicitly do not appear in the other class. 

In [9]:
only_pos_words = [word for word in pos_words if word not in neg_vocab]
only_neg_words = [word for word in neg_words if word not in pos_vocab]

opw_counter = Counter(only_pos_words)
onw_counter = Counter(only_neg_words)

print(opw_counter.most_common()[:50])
print('\n')
print(onw_counter.most_common()[:50])

[('Edie', 82), ('Gundam', 74), ('Antwone', 58), ('/>8/10', 47), ('/>7/10', 46), ('/>10/10', 45), ('Gunga', 44), ('Gypo', 44), ('Din', 43), ('Othello', 41), ('7/10.', 37), ('Blunt', 37), ('Yokai', 37), ('Tsui', 35), ('Blandings', 34), ('Goldsworthy', 32), ('/>9/10', 31), ('Gino', 31), ('Visconti', 30), ('Bernsen', 29), ('Taker', 29), ('Brashear', 29), ('Harilal', 29), ('Clutter', 28), ("Goldsworthy's", 27), ('"Rob', 26), ('Dominick', 25), ('MJ', 25), ('/>7', 24), ('Rosenstrasse', 24), ('Sassy', 24), ('Flavia', 24), ('Ashraf', 23), ('Recommended.', 22), ('Brock', 22), ('vulnerability', 22), ('Sabu', 22), ('Korda', 22), ('Ahmad', 22), ('Stevenson', 22), ('Coop', 22), ('Riff', 22), ('flawless.', 21), ('aunts', 21), ("Gilliam's", 21), ('Solo', 21), ('Kells', 21), ("Capote's", 21), ('Cutter', 21), ('Blackie', 21)]


[('/>4/10', 56), ('/>Avoid', 55), ('2/10', 49), ('*1/2', 45), ('unwatchable.', 43), ('/>3/10', 40), ('Thunderbirds', 40), ('Gamera', 39), ('steaming', 35), ('Wayans', 33), ('Slat

We begin to see some words we would expect to denote a negative review, but not so much for the positive reviews. Why might this be the case? What types of tokens are found in positive reviews but not in negative reviews?

*Your answer here*

The negative reivew words contain words such as "unwachable", "appalling", "incoherent" which are very indicative of negative sentiment. Positive review words contain far more proper nouns and names. This may be because positive reviews seem to give more of a description and summary of plot, rather than negative reviews are very to the point and explicit that the movie was bad. 


In [10]:
# Lets now make our combined vocabulary
space_vocab = list(pos_vocab.union(neg_vocab))
print('Length of space separated vocab: {}'.format(len(space_vocab)))
print(space_vocab[:50])

Length of space separated vocab: 281137
['unredeemable', 'lucid.)', 'CRYPT', 'Zumhofe', 'cry)...', 'MILD', 'style",', 'Bonzai"', 'answer?', 'stainless', 'disturbed,', 'intro.', 'Dorn,', 'France?', 'Golan,', 'comedy,,', 'contact?', 'donna', 'Rights-approved', 'thirdly...', '/>Later', 'psilcybe', ':C)', 'goofy,', 'shamelessly),', 'Andy', 'Killing', "Stack's,", "Meighan's", 'Arthurs', "Shahadah's", 'super-annoying.<br', 'scariest.', 'Speaksman', '"Boy,', 'DVD-R', "'adult'", 'coot,"', 'WALTER', 'lame-brain,', 'Vance', 'sun.Sheriff', 'Survives",', "night'.", 'MAJOR', 'Capote', 'Condemned.),and', 'Tad', 'Doran,', 'Bellwood.<br']


Looking at some words from our vocab, what issue do we find by only splitting on spaces?

*Your answer here*

One large issue we have by splitting only on spaces is that our vocab includes punctuation, single letters, text breakers, names, proper nouns, etc.

Now, rather than naively splitting on spaces, we can use tools which are informed about English grammar rules to create a cleaner tokenization.

In [11]:
from nltk.tokenize import word_tokenize

pos_examples_tokenized = [word_tokenize(ex) for ex in pos_examples]
neg_examples_tokenized = [word_tokenize(ex) for ex in neg_examples]

print(pos_examples_tokenized[0])

['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '``', 'Teachers', "''", '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '``', 'Teachers', "''", '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'line',

Looking at the first example we can see that things like apostrophes, periods, "n'ts" and ellipses are better handled.

Let's begin defining features for our model. The simplest features are simply if a word exists or not -- however, this is will be very slow if we decide to use the whole vocabulary. Instead, let's create these features for the top 100 most common words. 

In [12]:
all_tokenized_words = [word for ex in pos_examples_tokenized for word in ex] + \
    [word for ex in neg_examples_tokenized for word in ex]

atw_counter = Counter(all_tokenized_words)
top100 = [tup[0] for tup in atw_counter.most_common(100)] # A list of the top 100 most frequent word

print(top100)

['the', ',', '.', 'and', 'a', 'of', 'to', 'is', '/', '>', '<', 'br', 'in', 'I', 'it', 'that', "'s", 'this', 'was', 'The', 'as', 'with', 'movie', 'for', 'film', ')', '(', 'but', "n't", "''", '``', 'on', 'you', 'are', 'not', 'have', 'his', 'be', 'he', '!', 'one', 'at', 'by', 'all', 'an', 'who', 'they', 'from', 'like', 'It', 'her', 'so', 'or', 'about', 'has', 'just', 'out', '?', 'do', 'This', 'some', 'good', 'more', 'very', 'would', 'what', 'there', 'up', 'can', 'which', 'when', 'time', 'she', 'had', 'if', 'only', 'really', 'story', 'were', 'their', 'even', 'see', 'no', 'my', 'me', 'does', "'", 'did', ':', '-', 'than', '...', 'much', 'been', 'could', 'into', 'get', 'will', 'we', 'other']


In [13]:
####################
####################
##   Here I further clean the vocab to improve my model
#############

In [14]:
from nltk.corpus import stopwords

#Function to remove all stop words from the given word list such as "the", "a", "is", etc. 
def RemoveStopWords(word_list):
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    return filtered_words

In [16]:
pos_nostop = [RemoveStopWords(ex) for ex in pos_examples_tokenized[0:300]]##only using 300 since otherwise the computer overloads
neg_nostop = [RemoveStopWords(ex) for ex in neg_examples_tokenized[0:300]]

In [19]:
# Function to remove the proper nouns from a word list 
import nltk
from nltk import pos_tag
def RemoveProperNouns(word_list):
    tagged = nltk.tag.pos_tag(word_list)
    edited = [word for word,tag in tagged if tag != 'NNP' and tag != 'NNPS']
    return edited

In [20]:
pos_clean = [RemoveProperNouns(ex) for ex in pos_nostop]
neg_clean = [RemoveProperNouns(ex) for ex in neg_nostop]

In [21]:
all_clean_words = [word for ex in pos_clean for word in ex] + \
    [word for ex in neg_clean for word in ex]

all_clean_counter = Counter(all_clean_words)
top100clean = [tup[0] for tup in all_clean_counter.most_common(100)] # A list of the top 100 most frequent cleaned word

print(top100clean)

[',', '.', 'br', 'I', "'s", 'movie', 'The', ')', '(', 'film', "n't", "''", '``', 'one', '!', '<', 'like', 'It', '>', '?', 'good', 'This', 'story', 'would', 'time', 'even', '...', 'see', 'bad', 'much', ':', 'really', "'", 'get', 'first', '-', 'great', 'people', 'made', 'could', 'think', 'movies', 'seen', 'characters', 'well', 'make', 'ever', 'never', 'many', 'But', 'two', 'films', 'way', 'watch', '&', 'character', 'also', 'better', 'acting', 'scenes', 'plot', 'scene', 'In', 'go', 'little', ';', 'And', 'best', 'show', "'ve", 'actors', 'He', 'love', 'action', 'end', 'life', 'say', 'man', 'still', 'know', 'If', 'years', 'back', 'director', 'A', 'There', 'though', 'something', 'real', 'actually', 'got', 'pretty', 'watching', 'nothing', "'m", 'series', 'done', '--', 'give', 'thing']


Use the following block to define your own features for the NB model.

In [22]:
## I did the cleaning for this added feature above 
def top100_clean_word_features(example): # 100 features, 1 for each word in the top 100 most frequent cleaned words
    #note cleaned words has no stop words and no nouns
    return {word : 1 if word in example else 0 for word in top100clean}


##list of words that I would consider have a positive sentiment 
pos_sentiment=["fantastic", "wonderful", "incredible", "recommended"]
def example_feature(example): #Delete or modify this 
    return {word : 1 if word in example else 0 for word in pos_sentiment}

def create_feature_dictionary(example):
    features = {}
    for feat in [ top100_clean_word_features, example_feature]: #Once you've created your methods, and them to this list
        features.update(feat(example))
    return features


Now that we've defined our features for our model, we can create our final dataset, which will consist of extracted features and the example label. 

We'll also create a *validation* split by taking 20% of the training dataset. Remember, we never use the test set to make modeling decisions (in this case, decisions about features). Experiment with multiple models that make use of different combinations of features. Measure their performance on the validation split to figure out which features are the most helpful (use the show_most_informative_features function). When you've found your final model, evaluate its performance on the held out data. 

In [23]:
from nltk.classify import NaiveBayesClassifier
import random

# Convert training examples to a set of features.
train = [(create_feature_dictionary(ex), 0) for ex in neg_examples] + \
                [(create_feature_dictionary(ex), 1) for ex in pos_examples]

random.seed(42)
random.shuffle(train)

split_percent = .2

cutoff = int(split_percent * len(train))

validation_set = train[:cutoff]
training_set = train[cutoff:]

model = NaiveBayesClassifier.train(training_set)

In [24]:
from nltk.classify.util import accuracy

print('Validation accuracy: {}'.format(accuracy(model, validation_set)))
model.show_most_informative_features(10)

Validation accuracy: 0.7224
Most Informative Features
               wonderful = 1                   1 : 0      =      4.5 : 1.0
               fantastic = 1                   1 : 0      =      4.1 : 1.0
              incredible = 1                   1 : 0      =      3.1 : 1.0
             recommended = 1                   1 : 0      =      3.0 : 1.0
                     bad = 1                   0 : 1      =      2.9 : 1.0
                 nothing = 1                   0 : 1      =      2.1 : 1.0
                   great = 1                   1 : 0      =      2.0 : 1.0
                    love = 1                   1 : 0      =      1.8 : 1.0
                       ? = 1                   0 : 1      =      1.7 : 1.0
                    best = 1                   1 : 0      =      1.7 : 1.0


Describe the sets of features you've considered, and note down their performance below. What is the final set of features you found?

*Your answer here*

Since our goal is sentiment analysis, I wanted to evaluate the movies using words that actually indicat positive or negative. So I recleanedthe words to create a new feature. 

First, I removed all "stop words" such as “a”, “the”, “is”, “are”, etc. since these do not indicate any sentiment
Then I removed proper nouns
Lastly I took the top100 of that new set of clean words 

This brought the performance from ~61% to ~72%

Then,I also devised a list of words that I would classify as "positive sentiment" and that brought the validation accuracy to ~73%. Obviously, this list is just a start and with more research I could certainly improve this 


While I technically only added 2 features, the extraction of stop words and nouns took a significant amount of work and research and improved the accuracy considerably more than other ideas I tried (such as length). I hope I am not deducted points simply because I only added 2 features. I always believe in quality over quantity. 

Finally, test your model on the test set. 

In [25]:
# Load and process test data
pos_test_examples = read_folder(data_dir + 'test/pos/')
neg_test_examples = read_folder(data_dir + 'test/neg/')

test_set = [(create_feature_dictionary(ex), 0) for ex in neg_test_examples] + \
                [(create_feature_dictionary(ex), 1) for ex in pos_test_examples]

In [26]:
print('Test set accuracy: {}'.format(accuracy(model, test_set)))

# Note that we're looking at accuracy -- this is not always the most reliable metric and other choices like F1 might be more informative. 


Test set accuracy: 0.71948


In [27]:
## Self note: Total time spent on homework: 50+ hours. For next time- get debugging help/clarification earlier. 
#Ask sooner than later! 